In [18]:
import torch
import numpy as np
import torchaudio as ta
import sounddevice as sd
import matplotlib.pyplot as plt

doba = 10    # v sekundach

mylist = sd.query_devices()

def callback(outdata, frames, time, status):
    global current_sample, waveform_hovor, waveform_guitar, n_samples, sample_rate, a, f, wave
    if status:
        print(status)

    # Extract the next chunk of frames for both signals
    chunk_hovor = waveform_hovor[current_sample:current_sample+frames]
    chunk_guitar = waveform_guitar[current_sample:current_sample+frames]

    # Apply tremolo modulation only to the guitar chunk:
    n_vals = np.arange(current_sample, current_sample + len(chunk_guitar))
    envelope = (1 - a) + a * np.abs(np.sin(np.pi * f * (n_vals / sample_rate)))**wave
    envelope = envelope[:, np.newaxis]  # reshape for broadcasting (if multi-channel)
    modulated_chunk_guitar = chunk_guitar * envelope

    # If the chunks are shorter than the requested frames (at the end), pad with zeros.
    if len(chunk_hovor) < frames:
        pad = np.zeros((frames - len(chunk_hovor), chunk_hovor.shape[1]))
        chunk_hovor = np.vstack((chunk_hovor, pad))
    if len(modulated_chunk_guitar) < frames:
        pad = np.zeros((frames - len(modulated_chunk_guitar), modulated_chunk_guitar.shape[1]))
        modulated_chunk_guitar = np.vstack((modulated_chunk_guitar, pad))

    # Mix the two signals (add them together)
    mixed_chunk = chunk_hovor + 3*modulated_chunk_guitar

    # Write the mixed audio to the output buffer
    outdata[:] = mixed_chunk

    # Advance the global sample pointer
    current_sample += frames


In [20]:
# Paths to your audio files:
audio_path_hovor = './hovor_doprovod.wav'
audio_path_guitar = './hovor_guitar_flanger_reverb.wav'

# Load both audio files using torchaudio
waveform_hovor, sr_hovor = ta.load(audio_path_hovor)  # shape: [channels, samples]
waveform_guitar, sr_guitar = ta.load(audio_path_guitar)  # shape: [channels, samples]

# Ensure the sample rates match
if sr_hovor != sr_guitar:
    raise ValueError("Sample rates do not match!")
sample_rate = sr_hovor

# Transpose to shape [samples, channels] and convert to NumPy arrays
waveform_hovor = waveform_hovor.T.numpy()
waveform_guitar = waveform_guitar.T.numpy()

# Determine the number of samples to use (the minimum of the two)
n_samples = min(waveform_hovor.shape[0], waveform_guitar.shape[0])
current_sample = 0  # Global pointer

# Tremolo parameters for the guitar signal
a = 0.5  # modulation depth
f = 1     # modulation frequency in Hz (adjust as needed)
wave = 4  # ???????

# Create an OutputStream and play the mixed audio.
with sd.OutputStream(channels=waveform_hovor.shape[1],
                     samplerate=sample_rate,
                     callback=callback):
    # Keep the stream open for the duration of the shorter file.
    sd.sleep(int((n_samples / sample_rate) * 1000))

In [ ]:
# reverb + tremelo -> hnusne to zni
# lepsi: reverb + tremolo
# jsou to nelinearni efekty!
# tremolo neni casove invariatni -> nelinearni!

In [ ]:
### vibrato

# matblad zaklinadla:
# clc, clear all, close all

# [data, fs] = audioread('hovor_doprovod.wav');
# N = length(data);
# y = zeros(size(x));
#
# fr = 2
# depth = 100
# for n = 100:N-100
#    time = n/fs;
#    LFO = depth*sin(2*pi*fr*time) + 50; % +50 na chorus
#    M = fix(LFO);
#    gamma = LFO - M;
#    y(n,:) = data(n,:) + (1-gamma)*data(n-M,:) + gamma*data(n-M-1,:); % plus efekt chorus
#    % y(n,:) = (1-gamma)*data(n-M,:) + gamma*data(n-M-1,:); % metoda linearni interpolace
#    % y(n,:) data(n-round(LFO),:);
# end

# soundsc(y,fs)
#
# stereofonni efekt, slysitelny ve sluchatka hlavne, prijemny efekt:
# y(n,1) = data(n,1) + (1-gamma)*data(n-M,1) + gamma*data(n-M-1,1);
# y(n,2) = data(n,2) - (1-gamma)*data(n-M,2) - gamma*data(n-M-1,2);